<a href="https://colab.research.google.com/github/jyosa/evening/blob/main/old_demo.ipyn" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"

**Clone repository**

In [ ]:
!git clone https://github.com/jyosa/evening

Cloning into 'first-order-model'...
remote: Enumerating objects: 246, done.
remote: Total 246 (delta 0), reused 0 (delta 0), pack-reused 246
Receiving objects: 100% (246/246), 71.46 MiB | 22.54 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [ ]:
cd evening

/content/first-order-model


**Mount your Google drive folder on Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Let's initiate with one example, in the same folder you will find a video and an image as the file above, download the video and the image, then upload to the same folder in your google drive (first-order-model) and run the cell

In [ ]:
#model cell

import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/02.png')
reader = imageio.get_reader('/content/gdrive/My Drive/first-order-motion-model/04.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-adv-cpk.pth.tar')

**Perform image animation**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('/content/gdrive/My Drive/first-order-motion-model/generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from your folder first-order-motion-model in your drive

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:26<00:00,  7.92it/s]


The video will be without sound, if you want to use a sound like a voice, use the online source first extract the sound of the original video, next add it to the fake one. Then you can produce your own fake, recorded a video with your mobile in a horizontal way, the algorithm requires specific characteristics, so the first thing you have to do is to upload the video to the same drive directory and run the following script

In [ ]:
!ffmpeg -i /content/gdrive/My\ Drive/first-order-motion-model/your-video.mp4 -ss 00:00:05 -t 00:00:25 -filter:v "crop=600:600:760:50" -async 1 /content/gdrive/My\ Drive/first-order-motion-model/output.mp4

100%|██████████| 211/211 [00:26<00:00,  7.90it/s]


Change the name in the path your-video.mp4 to the correct name, avoid to gives names with spaces. The output will be placed in your google drive like output.mp4 that's the video you will use for running the algorithm. In the model cell, change the following path to the video and picture name eg if the video is called jy.mp4 and the image jy.png you must edit the following two lines as:

from 

source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/02.png') reader = imageio.get_reader('/content/gdrive/My Drive/first-order-motion-model/04.mp4')

to 

source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/jy.png') reader = imageio.get_reader('/content/gdrive/My Drive/first-order-motion-model/jy.mp4')

and run all the lines to get the video, the best fake video will win a spectacular Colombian coffee!!!!

Good look

